In [6]:
# Importamos librerias
import findspark
findspark.init()

import pyspark
from pyspark.sql.functions import *

from pyspark.sql import SparkSession

#Creamos la session
spark = (SparkSession
        .builder
        .appName("Nasa")
        .getOrCreate())

In [10]:
# Cargamos el archivo parquet con el que vamos a trabajar

nasa = spark.read.parquet("Nasa.parquet")

In [11]:
nasa.show()

+--------------------+------+--------------------+--------+------+-----+-------------------+
|                host|method|           resources|protocol|status| size|               time|
+--------------------+------+--------------------+--------+------+-----+-------------------+
|        199.72.81.55|   GET|    /history/apollo/|HTTP/1.0|   200| 6245|1995-07-01 00:00:01|
|unicomp6.unicomp.net|   GET| /shuttle/countdown/|HTTP/1.0|   200| 3985|1995-07-01 00:00:06|
|      199.120.110.21|   GET|/shuttle/missions...|HTTP/1.0|   200| 4085|1995-07-01 00:00:09|
|  burger.letters.com|   GET|/shuttle/countdow...|HTTP/1.0|   304|    0|1995-07-01 00:00:11|
|      199.120.110.21|   GET|/shuttle/missions...|HTTP/1.0|   200| 4179|1995-07-01 00:00:11|
|  burger.letters.com|   GET|/images/NASA-logo...|HTTP/1.0|   304|    0|1995-07-01 00:00:12|
|  burger.letters.com|   GET|/shuttle/countdow...|HTTP/1.0|   200|    0|1995-07-01 00:00:12|
|     205.212.115.106|   GET|/shuttle/countdow...|HTTP/1.0|   200| 398

In [13]:
nasa.select('host').count().distinct()

AttributeError: 'int' object has no attribute 'distinct'